In [1]:
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
#import logging
#import json
#import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

In [13]:
url='url'
filepath='C:/Users/Neon/Desktop/GitHub/download_entsoe/2017_2_AggregatedGenerationPerType.csv'
headers = ['region', 'variable', 'attribute', 'source', 'subsource','region_type','web']

In [18]:
def read_entsoe_transparency(filepath, url, headers):
    df = pd.read_csv(
        filepath,
        sep='\t',
        encoding='utf-16',
        header=0,
        index_col=['DateTime'],
        parse_dates=True,
        converters=[{'MapCode': lambda x: x[:2]},
                    {'AreaName': lambda x: x.replace(' CA','')},
                    {'AreaName': lambda x: x.replace(' BZ','')}],
        nrows=20,
        )
    
    df.drop(['year', 'month', 'day', 'AreaTypeCode', 'SubmissionTS'], 
            axis=1, inplace=True)
    df.rename(columns ={
        'ActualConsumption': 'consumption', 
        'ActualGenerationOutput': 'generation'},inplace=True)
    df.set_index(
        ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
        append=True, inplace=True)
    df=df.unstack('MapCode')
    df=df.unstack('ProductionType_Name')
    df=df.unstack('AreaName')
    df=df.unstack('AreaTypeCode')
    df=pd.concat(
        [df], 
        keys=['ENTSO-E'], 
        names=['source', 'attribute', 'region', 'variable', 'subsource'],
        axis=1)
    df=pd.concat([df], keys= url, names=['web'], axis=1)
    df=df.reorder_levels(headers, axis=1)
    
    return df

In [35]:
df = pd.read_csv(
    filepath,
    sep='\t',
    encoding='utf-16',
    header=0,
    index_col=['DateTime'],
    parse_dates=True,
    converters={'AreaName': lambda x: x.replace('\wCA',''),
                'MapCode': lambda x: x[:2],
                'AreaName': lambda x: x.replace(' CA',''),
                'AreaName': lambda x: x.replace(' BZ','')},
    nrows=20,
    )

df.drop(['year', 'month', 'day'], 
        axis=1, inplace=True)
df.rename(columns ={
    'ActualConsumption': 'consumption', 
    'ActualGenerationOutput': 'generation'},inplace=True)
df.set_index(
    ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
    append=True, inplace=True)
df=df.unstack('MapCode')
df=df.unstack('ProductionType_Name')
df=df.unstack('AreaName')
df=df.unstack('AreaTypeCode')
df=pd.concat(
    [df], 
    keys=['ENTSO-E'], 
    names=['source', 'attribute', 'region', 'variable', 'subsource','region_type'],
    axis=1)
df=pd.concat([df], keys= url, names=['web'], axis=1)
df=df.reorder_levels(headers, axis=1)

In [36]:
df.head()

region                       DE                                          \
variable               Biomass                Fossil Gas                  
attribute           consumption               consumption                 
source                  ENTSO-E                   ENTSO-E                 
subsource              MAVIR CA TransnetBW CA    MAVIR CA TransnetBW CA   
region_type                 CTA           CTA         CTA           CTA   
web                           u             u           u             u   
DateTime                                                                  
2017-02-01 10:00:00         NaN           NaN         NaN           NaN   
2017-02-01 11:45:00         NaN           NaN         NaN           NaN   
2017-02-01 12:15:00         NaN           NaN         NaN           NaN   
2017-02-01 18:15:00         NaN           NaN         NaN           NaN   
2017-02-10 01:00:00         NaN           NaN         NaN           NaN   

region                                         \
variable            Fossil Oil                  
attribute           consumption                 
source                  ENTSO-E                 
subsource              MAVIR CA TransnetBW CA   
region_type                 CTA           CTA   
web                           u             u   
DateTime                                        
2017-02-01 10:00:00         NaN           NaN   
2017-02-01 11:45:00         NaN           NaN   
2017-02-01 12:15:00         NaN           NaN   
2017-02-01 18:15:00         NaN           NaN   
2017-02-10 01:00:00         NaN           NaN   

region                                                              \
variable            Hydro Run-of-river and poundage                  
attribute                                consumption                 
source                                       ENTSO-E                 
subsource                                   MAVIR CA TransnetBW CA   
region_type                                      CTA           CTA   
web                                                u             u   
DateTime                                                             
2017-02-01 10:00:00                              NaN           NaN   
2017-02-01 11:45:00                              NaN           NaN   
2017-02-01 12:15:00                              NaN           NaN   
2017-02-01 18:15:00                              NaN           NaN   
2017-02-10 01:00:00                              NaN           NaN   

region                                                        ...       \
variable            Hydro Water Reservoir                     ...        
attribute                      consumption                    ...        
source                             ENTSO-E                    ...        
subsource                         MAVIR CA TransnetBW CA      ...        
region_type                            CTA           CTA      ...        
web                                      u             u      ...        
DateTime                                                      ...        
2017-02-01 10:00:00                    NaN           NaN      ...        
2017-02-01 11:45:00                    NaN           NaN      ...        
2017-02-01 12:15:00                    NaN           NaN      ...        
2017-02-01 18:15:00                    NaN           NaN      ...        
2017-02-10 01:00:00                    NaN           NaN      ...        

region                        HU                \
variable             Fossil Oil                  
attribute           SubmissionTS                 
source                   ENTSO-E                 
subsource               MAVIR CA TransnetBW CA   
region_type                  CTA           CTA   
web                            u             u   
DateTime                                         
2017-02-01 10:00:00         None          None   
2017-02-01 11:45:00         None          None   
2017-02-01 1